In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-13 20:26:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-05-13 20:26:27 (6.45 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

### Create DataFrames to match tables

In [5]:
# Import sql functions library
# https://stackoverflow.com/questions/70458086/is-there-way-to-import-all-pyspark-sql-functions
import pyspark.sql.functions as f



In [9]:
# Create the customers_table DataFrame, dropna and drop duplicate primary key rows
customers_df = df.groupby("customer_id").agg(f.count("customer_id").alias("customer_count")).dropna().drop_duplicates(["customer_id"])
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16747249|             4|
|     108460|             1|
|   11720077|            31|
|     437936|             1|
|     128131|             1|
|   39647402|             1|
|   13730111|             8|
|   43783459|             1|
|   12493167|           111|
|   40325486|            92|
|   44618558|             4|
|   17956020|             1|
|   10093406|             3|
|   40430762|             1|
|     134924|             1|
|   52427934|             1|
|   12879980|            15|
|   27680937|             7|
|   16186289|             1|
|   20326078|             1|
+-----------+--------------+
only showing top 20 rows



In [11]:
# Create the products_table DataFrame, dropna and drop duplicate primary key rows 
products_df = df.select(["product_id","product_title"]).dropna().drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001845357|    Ballad of Favour|
|0002168383|The Beatles: For ...|
|0002192365| Cage & Aviary Birds|
|0002216973|     Red Adam's Lady|
|0002230534|A single death: A...|
|0002234947|Miss Hobbema Pageant|
|0002261367|Unexpected Blessings|
|0006541151|Order Out of Chao...|
|0006544150|           In Xanadu|
|0006726372|I'll Go My Own Wa...|
|0006925227|The Vanishing Tre...|
|0007121040|Miss Marple's Fin...|
|0007155174|The Treasure Hunt...|
|000717201X|The Lord of the R...|
|0007178484|Reading "Lolita" ...|
|000718025X|Akbar and Jeff's ...|
|0007219040|The Secret Langua...|
|0007236093|  Dead Witch Walking|
|0007243626|The Yacoubian Bui...|
|0007255349|The Mobile Librar...|
+----------+--------------------+
only showing top 20 rows



In [12]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", f.to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).dropna().drop_duplicates(["review_id"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002YLZ8YP585|   16473634|B001KOTU92|     582225993| 2012-10-27|
|R1009E1Q7UILYO|   46872622|B004KABHQW|     559462795| 2013-01-11|
|R100EPYMF5J4CJ|    5913184|0071700285|     798175154| 2014-05-30|
|R100FP0C9S36QQ|   29790786|0684863065|     693395390| 2015-05-29|
|R100IOFNEWSFON|    8609387|0766861724|     265251973| 2014-08-01|
|R100JRX1JIYGB7|   20248524|076535697X|     524482751| 2014-02-21|
|R100NH1NIUXGN6|   44523714|9900493435|     638118828| 2014-10-04|
|R100Q2F8XEWNRJ|   13557200|0399158820|     130733609| 2013-04-27|
|R100V6BUO7PVRZ|   40193725|0679722874|     693721236| 2015-01-21|
|R100VPICZRNDCA|    5145795|1634431367|     493632375| 2015-06-15|
|R100XWWX1YHY2N|   52249576|0789210649|     350767168| 2013-05-25|
|R1014LYBMPQPLV|   46529674|1476731934|     126926034| 2014-11

In [13]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"]).dropna().drop_duplicates(["review_id"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1002YLZ8YP585|          1|            6|         20|   N|                Y|
|R1009E1Q7UILYO|          4|            2|          2|   N|                Y|
|R100EPYMF5J4CJ|          5|            0|          0|   N|                Y|
|R100FP0C9S36QQ|          5|            0|          0|   N|                Y|
|R100IOFNEWSFON|          5|            0|          0|   N|                Y|
|R100JRX1JIYGB7|          5|            0|          0|   N|                Y|
|R100NH1NIUXGN6|          5|            0|          0|   N|                Y|
|R100Q2F8XEWNRJ|          5|            0|          0|   N|                Y|
|R100V6BUO7PVRZ|          5|            8|          9|   N|                Y|
|R100VPICZRNDCA|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [14]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.czgsfb9razm2.us-east-2.rds.amazonaws.com:5432/book_review_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored